In [1]:
#cv2 : computer vision library  
import cv2 
import face_recognition

In [13]:
# Load the image of the student
Rameez = face_recognition.load_image_file('C:/Users/DELL/Pictures/1702399592320.jpeg')
# Detect and encode the face (generate embeddings)
face_encoding_rameez = face_recognition.face_encodings(Rameez)[0]
# Load and encode a new image
Umair = face_recognition.load_image_file("C:/Users/DELL/Pictures/Camera Roll/download.jpeg")
face_encoding_umair = face_recognition.face_encodings(Umair)[0]


In [ ]:
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient('localhost', 27017)
db = client['student_database']
students = db['students']
attendance = db['attendance'] 

In [ ]:
#add entry of new student on mongoDb database along with his/her face encoding for face detection
sample_student = {
    "name": "Rameez",
    "semester": 7,
    "age": 21,
    "gender": "male",
    "face_encoding": face_encoding_rameez.tolist()
}
# This will create the 'students' collection if it doesn't exist
students.insert_one(sample_student)


In [17]:
# first i try face recognition system by providing image path and then below cell i convert this in live face detection
# Load a new image and encode the face
unknown_image = face_recognition.load_image_file("C:/Users/DELL/Pictures/1702399592320.jpeg")
unknown_face_encoding = face_recognition.face_encodings(unknown_image)[0]

all_students = students.find()
found_match = False

for student in all_students:
    known_face_encoding = student['face_encoding'] 
    # Compare the unknown face with known faces using compare_faces method
    result = face_recognition.compare_faces([known_face_encoding], unknown_face_encoding)
    if result[0]:
        print(f"Match found: {student['name']}, {student['semester']}")
        found_match = True
        attendance.update_one(
          {'student_id': student['_id'], 'date': '2024-11-04'},  # Use current date
          {'$set': {'present': True}}, 
          upsert=True  # Create entry if it doesn't exist
        )
        print(f"Attendance recorded for {student['name']}")
        break

if not found_match:
    print("unknown")



Match found: Rameez, 7
Attendance recorded for Rameez


In [58]:
# live face recognition

import numpy as np

# Initialize webcam
video_capture = cv2.VideoCapture(0)

# Capture a single frame to use as the "unknown" face
ret, frame = video_capture.read()

# Convert the frame to RGB (face_recognition library expects RGB format)
rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

# Detect face locations in the frame
face_locations = face_recognition.face_locations(rgb_frame)

# Detect and encode the face(s) in the frame using the face locations
unknown_face_encodings =np.array(face_recognition.face_encodings(rgb_frame, face_locations))

# Initialize variable to track if a match is found
found_match = False

# Retrieve all student records (assuming `students` is your MongoDB collection)
all_students = students.find()

# Loop through all students to compare known encodings with the unknown face
for student in all_students:
    known_face_encoding = student['face_encoding'] 
            
            # Compare the unknown face with each student's known face encoding
    result = face_recognition.compare_faces([known_face_encoding], unknown_face_encodings)
            
    if result[0]:  # If a match is found
        print(f"Match found: {student['name']}, {student['semester']}")
        found_match = True
                
                # Record attendance in MongoDB
        attendance.update_one(
            {'student_id': student['_id'], 'date': '2024-11-05'},  # Replace with current date as needed
            {'$set': {'present': True}}, 
            upsert=True  # Create entry if it doesn't exist
        )
        print(f"Attendance recorded for {student['name']}")
        break  # Exit loop on finding a match

if found_match == False:
    print("Unknown Face Detected")

# Display the captured frame in a window
cv2.imshow('Real-Time Face Recognition', frame)

# Release the webcam and close any OpenCV windows
video_capture.release()
cv2.destroyAllWindows()


Match found: Rameez, 7
Attendance recorded for Rameez
No face detected in the webcam frame.


In [9]:
all_students = students.find()  # This retrieves all documents

# Loop through the results and print each record
for student in all_students:
    print(f"Name: {student['name']}, Semester: {student['semester']}, Age: {student['age']}, Gender: {student['gender']}")

Name: Umair, Semester: 3, Age: 24, Gender: male
Name: Rameez, Semester: 7, Age: 21, Gender: male


In [6]:
#delete_result = students.delete_one({'name': 'Rameez'})

In [12]:
# Fetch all attendance records
all_records = attendance.find()
for record in all_records:
    print(record)


{'_id': ObjectId('672636d8ad62c101c634cc41'), 'date': '2024-10-05', 'student_id': ObjectId('67168736006e5bbd6332fbbb'), 'present': True}
{'_id': ObjectId('672637e3ad62c101c634cc7e'), 'student_id': ObjectId('6716851a006e5bbd6332fbb8'), 'date': '2024-10-05', 'present': True}
{'_id': ObjectId('67264215ad62c101c634cec6'), 'date': '2024-11-02', 'student_id': ObjectId('6716851a006e5bbd6332fbb8'), 'present': True}
{'_id': ObjectId('672a4e00ad62c101c634d687'), 'date': '2024-11-03', 'student_id': ObjectId('6716851a006e5bbd6332fbb8'), 'present': True}
{'_id': ObjectId('672a542aad62c101c634d7dc'), 'date': '2024-11-05', 'student_id': ObjectId('6716851a006e5bbd6332fbb8'), 'present': True}
{'_id': ObjectId('672a58b1ad62c101c634d8d8'), 'date': '2024-11-05', 'student_id': ObjectId('67168736006e5bbd6332fbbb'), 'present': True}
{'_id': ObjectId('672b6db1ad62c101c634d9a5'), 'date': '2024-11-04', 'student_id': ObjectId('6716851a006e5bbd6332fbb8'), 'present': True}
{'_id': ObjectId('672b6f10ad62c101c634d9f

In [8]:
# get the attendance of specific student

from bson import ObjectId

student_id = '6716851a006e5bbd6332fbb8'
student_id = ObjectId(student_id)

# Query all attendance records for this student
student_records = attendance.find({'student_id': student_id})

# Display each attendance record for the student
for record in student_records:
    print(record)



{'_id': ObjectId('672637e3ad62c101c634cc7e'), 'student_id': ObjectId('6716851a006e5bbd6332fbb8'), 'date': '2024-10-05', 'present': True}
{'_id': ObjectId('67264215ad62c101c634cec6'), 'date': '2024-11-02', 'student_id': ObjectId('6716851a006e5bbd6332fbb8'), 'present': True}
{'_id': ObjectId('672a4e00ad62c101c634d687'), 'date': '2024-11-03', 'student_id': ObjectId('6716851a006e5bbd6332fbb8'), 'present': True}
{'_id': ObjectId('672a542aad62c101c634d7dc'), 'date': '2024-11-05', 'student_id': ObjectId('6716851a006e5bbd6332fbb8'), 'present': True}
{'_id': ObjectId('672b6db1ad62c101c634d9a5'), 'date': '2024-11-04', 'student_id': ObjectId('6716851a006e5bbd6332fbb8'), 'present': True}


In [32]:
# get the id of specific student, then use it to get that student attendance
student = students.find_one({"name": "Umair"})
if student:
    student_id = student['_id']
    print(f"Student ID: {student_id}")
else:
    print("Student not found.")


Student ID: 6716851a006e5bbd6332fbb8
